In [40]:
from dotenv import load_dotenv
import os
import pygsheets
import requests
import pandas as pd
from tqdm import tqdm

from pangres import upsert
from sqlalchemy import text, create_engine

load_dotenv()

True

In [70]:
db_username=os.environ.get("db_username")
db_password=os.environ.get("db_password")
db_host=os.environ.get("db_host")
db_port=os.environ.get("db_port")
# db_name=os.environ.get("db_name")
db_name = 'lol_analytics'


def create_db_connection_string(db_username: str, db_password: str, db_host: str, db_port: int, db_name: str):
    connection_url = f"postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"
    return connection_url

connection_url = create_db_connection_string(db_username, db_password, db_host, db_port, db_name)
db_engine = create_engine(connection_url, pool_recycle=3600)
connection = db_engine.connect()

## TESTING Retrieve API information

In [2]:
def get_puuid(gameName, tagline, api_key):
    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{username}/{tagline}?api_key={api_key}'

    response = requests.get(link)
    if response.status_code == 200:
        return response.json()['puuid']
    else:
        print(f"Error: {response.status_code}")
    return None

In [3]:
def account_info_by_puuid(puuid, api_key):
    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}'

    response = requests.get(link)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
    return None

In [4]:
def retrieve_match_ids(puuid, api_key, type='ranked'):
    link = f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type={type}&start=0&count=100&api_key={api_key}'

    response = requests.get(link)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
    return None

In [5]:
def retrieve_match_data(match_id, api_key):
    link = f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
    response = requests.get(link)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
    return response.json()

In [ ]:
def process_match_json(match_id, puuid, api_key):
    game = retrieve_match_data(match_id, api_key)

    metadata = game['metadata']
    data_version = metadata['dataVersion']
    match_id = metadata['matchId']
    participants_puuid = metadata['participants']

    match_info = game['info']
    match_duration = match_info['gameDuration'] # 
    match_mode = match_info['gameMode']
    match_type = match_info['gameType']
    match_version = match_info['gameVersion']
    match_map_id = match_info['mapId']
    match_queue_id = match_info['queueId']
    participant_data = match_info['participants']


    main_player = participant_data[participants_puuid.index(puuid)]
    champion_level = main_player['champLevel']
    champion_experience = main_player['champExperience']
    champion_name = main_player['championName']
    champion_id = main_player['championId']
    deaths = main_player['deaths']
    kills = main_player['kills']
    assists = main_player['assists']
    gold_earned = main_player['goldEarned']
    gold_spent = main_player['goldSpent']
    individual_position = main_player['individualPosition']
    item_0 = main_player['item0']
    item_1 = main_player['item1']
    item_2 = main_player['item2']
    item_3 = main_player['item3']
    item_4 = main_player['item4']
    item_5 = main_player['item5']
    item_6 = main_player['item6']
    lane = main_player['lane']
    neutral_minions_killed = main_player['neutralMinionsKilled']
    objectives_stolen = main_player['objectivesStolen']
    participant_id = main_player['participantId']
    riot_id_game_name = main_player['riotIdGameName']
    riot_id_tagline = main_player['riotIdTagline']
    role = main_player['role']
    total_damage_dealt = main_player['totalDamageDealt']
    total_damage_dealt_to_champions = main_player['totalDamageDealtToChampions']
    total_damage_shielded_on_teammates = main_player['totalDamageShieldedOnTeammates']
    total_damage_taken = main_player['totalDamageTaken']
    totalHealsOnTeammates = main_player['totalHealsOnTeammates']
    total_minions_killed = main_player['totalMinionsKilled']
    total_time_cc_dealt = main_player['totalTimeCCDealt']
    team_id = main_player['teamId']
    team_position = main_player['teamPosition']
    turret_kills = main_player['turretKills']
    vision_score = main_player['visionScore']
    vision_wards_bought = main_player['visionWardsBoughtInGame']
    wards_placed = main_player['wardsPlaced']
    wards_killed = main_player['wardsKilled']
    win = main_player['win']

    # perks
    main_player_perks = main_player['perks']
    stats_perks = main_player_perks['statPerks']
    perk_defense = stats_perks['defense']
    perk_flex = stats_perks['flex']
    perk_offense = stats_perks['offense']

    perk_style_selection = main_player_perks['styles']

    primary_style = perk_style_selection[0]
    primary_style_selections = primary_style['selections']
    keystone = primary_style_selections[0]['perk']
    primary_style_selection_1 = primary_style_selections[1]['perk']
    primary_style_selection_2 = primary_style_selections[2]['perk']
    primary_style_selection_3 = primary_style_selections[3]['perk']

    secondary_style = perk_style_selection[1]
    secondary_style_selections = secondary_style['selections']
    secondary_style_selection_1 = secondary_style_selections[0]['perk']
    secondary_style_selection_2 = secondary_style_selections[1]['perk']


    for team in match_info['teams']:
        team_id = team['teamId']
        team_win = team['win']
        bans = team['bans']

        if bans:
            for ban in bans:
                champion_id = ban['championId']
                # champion_name = ban['championName']


        objectives = team['objectives']
        if 'atakhan' in objectives:
            atakhan = objectives['atakhan']
        else:
            atakhan = 0
        dragon = objectives['dragon']
        inhibitor = objectives['inhibitor']
        rift_herald = objectives['riftHerald']
        tower = objectives['tower']
        baron = objectives['baron']
        horde = objectives['horde']
        champion = objectives['champion']


    match_dataframe = pd.DataFrame(
        {
            'data_version': [data_version],
            'match_id': [match_id],
            'puuid': [puuid],
            'participants_puuid': [participants_puuid],
            'match_mode': [match_mode],
            'match_type': [match_type],
            'match_duration': [match_duration],
            'match_version': [match_version],
            'match_map_id': [match_map_id],
            'match_queue_id': [match_queue_id],
            'champion_level': [champion_level],
            'champion_experience': [champion_experience],
            'champion_name': [champion_name],
            'champion_id': [champion_id],
            'deaths': [deaths],
            'kills': [kills],
            'assists': [assists],
            'gold_earned': [gold_earned],
            'gold_spent': [gold_spent],
            'individual_position': [individual_position],
            'item_0': [item_0],
            'item_1': [item_1],
            'item_2': [item_2],
            'item_3': [item_3],
            'item_4': [item_4],
            'item_5': [item_5],
            'item_6': [item_6],
            'lane': [lane],
            'neutral_minions_killed': [neutral_minions_killed],
            'objectives_stolen': [objectives_stolen],
            'participant_id': [participant_id],
            'riot_id_game_name': [riot_id_game_name],
            'riot_id_tagline': [riot_id_tagline],
            'role': [role],
            'total_damage_dealt': [total_damage_dealt],
            'total_damage_dealt_to_champions': [total_damage_dealt_to_champions],
            'total_damage_shielded_on_teammates': [total_damage_shielded_on_teammates],
            'total_damage_taken': [total_damage_taken],
            'totalHealsOnTeammates': [totalHealsOnTeammates],
            'total_minions_killed': [total_minions_killed],
            'total_time_cc_dealt': [total_time_cc_dealt],
            'team_id': [team_id],
            'team_position': [team_position],
            'turret_kills': [turret_kills],
            'vision_score': [vision_score],
            'vision_wards_bought': [vision_wards_bought],
            'wards_placed': [wards_placed],
            'wards_killed': [wards_killed],
            'win': [win],
            'keystone': [keystone],
            'primary_style_selection_1': [primary_style_selection_1],
            'primary_style_selection_2': [primary_style_selection_2],
            'primary_style_selection_3': [primary_style_selection_3],
            'secondary_style_selection_1': [secondary_style_selection_1],
            'secondary_style_selection_2': [secondary_style_selection_2],
            'perk_defense': [perk_defense],
            'perk_flex': [perk_flex],
            'perk_offense': [perk_offense],
            'team_id': [team_id],
            'team_win': [team_win],
            'atakhan': [atakhan],
            'dragon': [dragon],
            'inhibitor': [inhibitor],
            'rift_herald': [rift_herald],
            'tower': [tower],
            'baron': [baron],
            'horde': [horde],
            'champion': [champion],
        }
    )
    return match_dataframe

In [ ]:
username = 'oanduh'
tagline = 'oan'

api_key = os.getenv("riot_api_key")
puuid = get_puuid(username, tagline, api_key)

account_info_by_puuid(puuid, api_key)
match_ids = retrieve_match_ids(puuid, api_key)

dataframe_list = []

for match_id in tqdm(match_ids):
    match_dataframe = process_match_json(match_id, puuid, api_key)
    dataframe_list.append(match_dataframe)



 97%|█████████▋| 97/100 [00:28<00:00,  3.41it/s]


Error: 429


KeyError: 'metadata'

In [45]:
all_matches_dataframe = pd.concat(dataframe_list)

In [46]:
def retrieve_perk_info():
    perks = requests.get('https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perks.json').json()
    perkstyles = requests.get('https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perkstyles.json').json()
    
    perk_dict = {}
    for i in perkstyles['styles']:
        perk_dict[i['id']] = i['name']

    for i in perks:
        perk_dict[i['id']] = i['name']
    return perk_dict

In [47]:
perk_dict = retrieve_perk_info()

In [48]:
perk_columns = ['keystone', 'primary_style_selection_1', 'primary_style_selection_2', 
                'primary_style_selection_3', 'secondary_style_selection_1', 
                'secondary_style_selection_2', 'perk_defense', 'perk_flex', 'perk_offense']

all_matches_dataframe[perk_columns] = all_matches_dataframe[perk_columns].replace(perk_dict)

In [80]:
all_matches_dataframe['username'] = [username]*len(all_matches_dataframe)
all_matches_dataframe['tagline'] = [tagline]*len(all_matches_dataframe)

In [ ]:
# all_matches_dataframe['puuid'] = [puuid]*len(all_matches_dataframe)
all_matches_dataframe['uuid'] = all_matches_dataframe['match_id'] + '_' + all_matches_dataframe['puuid']
all_matches_dataframe = all_matches_dataframe.set_index('uuid')

In [81]:
upsert(con=connection, df=all_matches_dataframe, schema='soloq', table_name='player_matches', create_schema=True, if_row_exists='update')

In [82]:

connection.commit()

In [83]:
with db_engine.connect() as connection:
    df = pd.read_sql(text("SELECT * FROM soloq.player_matches"), connection)


In [84]:
main_lane_games = df[df['lane'] == 'MIDDLE']
main_lane_games['champion_name'].value_counts()

champion_name
Hwei           11
Syndra          8
Mel             7
Ryze            4
Akshan          4
TwistedFate     4
Akali           3
Smolder         3
Aurora          3
Sylas           2
Viktor          2
Yone            2
Naafiri         2
Tristana        1
Morgana         1
Jayce           1
Ahri            1
Taliyah         1
Kassadin        1
Malzahar        1
Name: count, dtype: int64

In [ ]:
mains = main_lane_games['champion_name'].value_counts()[:3].reset_index()['champion_name'].tolist()

In [ ]:
mains_only = main_lane_games[main_lane_games['champion_name'].isin(mains)]
mains_only['champion_name'].value_counts()

champion_name
Hwei      10
Syndra     9
Mel        6
Name: count, dtype: int64

In [ ]:
dragon = mains_only.set_index('champion_name')['dragon']
dragon_list = []
for i, row in enumerate(dragon):
    dragon_list.append(row)

dragon_df = pd.DataFrame(dragon_list)

In [ ]:
dragon = dragon.reset_index()
dragon['dragon'] = dragon_df['kills']
dragon['first_kill'] = dragon_df['first']

In [ ]:
dragon.groupby('champion_name').mean()

,dragon,first_kill
champion_name,,
Hwei,2.300000,0.600000
Mel,2.166667,0.833333
Syndra,1.888889,0.444444


In [ ]:
summarized_stats = mains_only.groupby('champion_name')[['match_duration', 'champion_level','turret_kills', 'total_damage_dealt_to_champions', 'wards_killed','wards_placed', 'gold_earned', 'total_minions_killed', 'kills', 'deaths','assists', 'win']].mean()

summarized_stats['match_duration_in_minutes'] = summarized_stats['match_duration']/60
summarized_stats['cs_per_minute'] = summarized_stats['total_minions_killed']/summarized_stats['match_duration_in_minutes']
summarized_stats

,match_duration,champion_level,turret_kills,total_damage_dealt_to_champions,wards_killed,wards_placed,gold_earned,total_minions_killed,kills,deaths,assists,win,match_duration_in_minutes,cs_per_minute
champion_name,,,,,,,,,,,,,,
Hwei,1854.000000,16.100000,0.700000,28943.000000,2.200000,12.900000,12133.300000,197.600000,6.600000,4.100000,12.000000,0.600000,30.900000,6.394822
Mel,1777.166667,15.500000,1.000000,32042.666667,2.166667,11.500000,14141.666667,191.333333,13.333333,4.666667,7.000000,0.500000,29.619444,6.459721
Syndra,1898.777778,15.777778,0.777778,28415.111111,1.555556,13.333333,12187.000000,200.777778,5.333333,6.111111,5.222222,0.333333,31.646296,6.344432


In [ ]:
# need to create 2 databases? maybe retrieve as much data as possible for challenger ladder?
# how would we update something so frequent consideirng that to compare stats we would need constant updating 
# maybe user windows scheduler to update the live db?



